Here we have 3 bots - Gpt, Gemini and Claude. They are trying to find the top 3 stocks on NYSE which all the 3 bots identify, review and finalize when in consensus. The call to the model is kept with a range of 100 in for loop to mimic infinite loop as a breaking condition is set for when the 3 bots are in consensus for the top 3 stocks.
I would like to invite the reader to go through the code and share any feedbacks that could help me improve more on this. Any suggestions and feedbacks are most welcome. You could send your feedback at - srbmisc@gmail.com.

Thank You

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
openai = OpenAI()

In [ ]:
sp_gpt = '''You are a bot Gpt. You are in a conversation with 2 other bots Gemini and Claude. All 3 of you are trying to figure out the top 3 stocks that have performed so far in NYSE. 
At every turn, you propose a stock, share its performance and fundamentals, and ask for the others for their review on this stock. Similarly, when its their turn and they share their stock pick, you review their pick.
If you think thier pick was better, accept it and in your next turn share that same stock otherwise ask them to accept your pick. The goal is to come up with 3 stocks at the end that all 3 participants consider the best.
If there is a concensus on 3 top stocks and its your turn, just output like this CONSENSUS REACHED : stock 1, stock 2, stock 3
Prefix your response with Gpt: in bold and respond in Markdown'''

sp_gemini = '''You are a bot Gemini. You are in a conversation with 2 other bots Gpt and Claude. All 3 of you are trying to figure out the top 3 stocks that have performed so far in NYSE. 
At every turn, you propose a stock, share its performance and fundamentals, and ask for the others for their review on this stock. Similarly, when its their turn and they share their stock pick, you review their pick.
If you think thier pick was better, accept it and in your next turn share that same stock otherwise ask them to accept your pick. The goal is to come up with 3 stocks at the end that all 3 participants consider the best.
If there is a concensus on 3 top stocks and its your turn, just output like this CONSENSUS REACHED : stock 1, stock 2, stock 3
Prefix your response with Gemini: in bold and respond in Markdown'''

sp_claude = '''You are a bot Claude. You are in a conversation with 2 other bots Gemini and Gpt. All 3 of you are trying to figure out the top 3 stocks that have performed so far in NYSE. 
At every turn, you propose a stock, share its performance and fundamentals, and ask for the others for their review on this stock. Similarly, when its their turn and they share their stock pick, you review their pick.
If you think thier pick was better, accept it and in your next turn share that same stock otherwise ask them to accept your pick. The goal is to come up with 3 stocks at the end that all 3 participants consider the best.
If there is a concensus on 3 top stocks and its your turn, just output like this CONSENSUS REACHED : stock 1, stock 2, stock 3
Prefix your response with Claude: in bold and respond in Markdown'''

talk = "Gpt: Hello Gemini, Hello Claude. I want to discuss with you a good stock on the NYSE with you.<br> Gemini: Hello Gpt, Hello Claude. Sure go ahead, give us the best stock you think is there on the NYSE ?<br> Claude: Hello Gpt, Hello Gemini. Sure Gpt, lets discuss on some stocks. What stock do you have on mind ?<br>"



In [ ]:
def callBot(mode):

    global talk
    talk = talk + "<br><br><br>"
    messages = [{"role": "system", "content": sp_gpt if mode==0 else (sp_gemini if mode==1 else sp_claude)},
    {"role":"user", "content":talk}]

    if mode==0:
        model = 'gpt-4.1-mini'
        client = OpenAI()
    elif mode==1:
        model = 'gemini-2.5-flash'
        client = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
    else:
        model = 'claude-3-5-haiku-latest'
        client = OpenAI(api_key=anthropic_api_key, base_url="https://api.anthropic.com/v1/")

    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True
    )
    for chunk in stream:
        talk += (chunk.choices[0].delta.content or '')
        talk = talk.replace("```","").replace("markdown","")
        update_display(Markdown(talk), display_id=display_handle.display_id)

    

In [ ]:
display_handle = display(Markdown(""), display_id=True) 

for i in range(100):
    callBot(i%3)
    if 'CONSENSUS REACHED :' in talk or 'CONSENSUS REACHED:' in talk:
        break

   